Any cell that should be run everytime will have * in the top comment/line

near future upgrades :  
**implement limit of number of succesful licks (need to track licks first thooo**  
**alert when done, make it easy to see how much time has passed/is left**  
 *move arduino to opposite side of the rig to enable camera placement on user side *  
also i think we could make the lines specifically print around solenoid activated bc thats what i care about seeing real time (ie is it licking?)
   
resolved upgrade: buffered write to txt file (dont want to add delay if not needed    
how do you determine what the buffer size should be and what are the units ?      
caffeinate/prevent sleep based on OS. 
  
notes:   
it actuaLLY matters if mouse is holding bar when behavior starts, so we need to start behvaior before the bar is placed  
otherwise the bar touch sensor will not work until it slowly recalibrates itself   
  

tomorrow morning work on the damn camera system with atlas

Press esc M to change cell to text (markdown)
To stop cell, click outside of code section and press II quickly (not capital)   
To keyboard interrupt (ie stop cell stop behavior) click the leftside bar but outside the cell  
Press ii quickly

In [16]:
# * run at start of each session, clear all variables
for var in list(globals().keys()):
    if var not in ['__builtins__', '__name__', '__doc__', '__package__', '__loader__', '__spec__', '__annotations__']:
        del globals()[var]

In [17]:
import serial
import time 
from datetime import datetime
import os
import csv
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askdirectory

(**mac**) To get port number type 'ls /dev/tty.*' in terminal

In [3]:
# get port info, on pc port should just be something like COM3
import serial.tools.list_ports
ports = serial.tools.list_ports.comports()
for port in ports:
    print(f"Port: {port.device}, Path: {port.hwid}, Description: {port.description}")

Port: COM3, Path: USB VID:PID=2341:0043 SER=851303035383512002A0 LOCATION=1-1, Description: Arduino Uno (COM3)
Port: COM1, Path: ACPI\PNP0501\0, Description: Communications Port (COM1)


In [18]:
#* caffeine!!! run to prevent sleep / interruption of data collection
import platform
import subprocess
import ctypes

os_name = os.name
if os_name =="nt":
    print("This is a windows system.")
   # Use ctypes to prevent sleep on Windows 
    ctypes.windll.kernel32.SetThreadExecutionState( 
        0x80000000 | 0x00000001 | 0x00000002 ) 
    print("Sleep mode disabled on Windows.")
elif os_name == "posix":
    if platform.system() == "Darwin":
        print("This is a macOS system.")
        caffeinate = subprocess.Popen(['caffeinate'])
        print("Caffeinate activated on macOS.")
    else:
        print("this is a Linux system.")

This is a windows system.
Sleep mode disabled on Windows.


In [25]:
# * run this cell to test behavior rig for run_time
port = 'COM3'  # Replace with your Arduino's port
baudrate = 115200  # make sure this matches the baud rate in the Arduino sketch (.ino file)

#request user for identiying information
animal_ID = input("Enter Animal ID: ")
training_stage = input("Enter training stage: ")
min_ = float(input("Session length (min): "))
run_time = min_*60 #sec

#open window to choose directory 
root = Tk()
root.withdraw()  # Hide the main tkinter window
save_directory = askdirectory(title="Select Directory to Save Data File")
data_filename = f"{animal_ID}_{training_stage}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
data_filepath = os.path.join(save_directory, data_filename)

#initialize file to write to as 'file'
with open(data_filepath, 'w') as file:
    #initialize serial connection
    ser = serial.Serial(port, baudrate, timeout=1)
    time.sleep(2) 
    #TRIGGER CAMERA, 2P HERE
    
    #send start command to Arduino
    start_time = time.time() #use for duration based arduino script start/stop
    ser.write(b'S') 
    behav_active = True
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    print(f'{timestamp} - Sent start command to Arduino.')
    file.write(f'{timestamp} - Sent start command to Arduino.\n')
    
    try:
        count = 0
        # Read data from Arduino
        while behav_active:
            if ser.in_waiting > 0:
                try:
                    line = ser.readline().decode('utf-8', errors='ignore').rstrip()
                    timestamp = datetime.now().strftime('%H:%M:%S.%f')[:-3]
                    log_line = f'{timestamp} - Arduino: {line}\n'
                    # Reduce print statements to every 10th message
                    count += 1
                    if count % 10 == 0:
                        print(log_line, end='')
                    #print(log_line, end='')  # Print the data in real-time
                    file.write(log_line)  # file.write() has an auto determined buffer size 
                    ##can implement specific buffering by calling the lines below every 100 measurements for example
                    #if count % 100 ==0:    
                        #file.flush()  # Ensure the data is written to the disk
                        #os.fsync(file.fileno())  # Force writing to disk
                except UnicodeDecodeError:
                    print("Failed to decode serial data")
                    continue
    
            # Check if 20 seconds have passed to send the stop command
            if time.time() - start_time >= run_time:
                #execution line (send stop command to arduino)
                ser.write(b'X') 
                #reporting lines
                print('Sent stop command to Arduino.')
                timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
                file.write(f'{timestamp} - Sent stop command to Arduino.')
                #exit behavior code line
                behav_active = False
    
    except KeyboardInterrupt:
        #execution line (send stop command to arduino)
        ser.write(b'X')
        #reporting lines
        print("Interrupted by user. Sent stop command to Arduino.")
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        file.write(f'{timestamp} - Interrupted by user. Sent stop command to Arduino.')
        file.flush()
        os.fsync(file.fileno())
        behav_active = False
    finally:
        #close serial connection
        ser.close()

2024-09-10 20:55:40.672 - Sent start command to Arduino.
20:55:40.692 - Arduino: bar: 0
20:55:40.709 - Arduino: lick: 8
20:55:40.729 - Arduino: Solenoid pin state: 0
20:55:40.746 - Arduino: bar: 0
20:55:40.765 - Arduino: lick: 7
20:55:40.782 - Arduino: Solenoid pin state: 0
20:55:40.802 - Arduino: bar: 0
20:55:40.819 - Arduino: lick: 7
20:55:40.836 - Arduino: Solenoid pin state: 0
20:55:40.855 - Arduino: bar: 1
20:55:40.872 - Arduino: lick: 7
20:55:40.893 - Arduino: Solenoid pin state: 0
20:55:40.909 - Arduino: bar: 1
20:55:40.930 - Arduino: lick: 7
20:55:40.946 - Arduino: Solenoid pin state: 0
20:55:40.966 - Arduino: bar: 1
20:55:40.982 - Arduino: lick: 7
20:55:41.000 - Arduino: Solenoid pin state: 0
20:55:41.019 - Arduino: bar: 1
20:55:41.035 - Arduino: lick: 7
20:55:41.056 - Arduino: Solenoid pin state: 0
20:55:41.073 - Arduino: bar: 1
20:55:41.093 - Arduino: lick: 7
20:55:41.110 - Arduino: Solenoid pin state: 0
20:55:41.130 - Arduino: bar: 0
20:55:41.146 - Arduino: lick: 7
20:55:41

In [14]:
#**ALWAYS RUN to dump melatonin on the coffee jitter 
#jk but this cell is v important to reenable the computer running script to sleep
import platform
import subprocess
import ctypes
os_name = os.name
if os_name =="nt":
    print("This is a windows system.")
    # Re-enable sleep mode on Windows 
    ctypes.windll.kernel32.SetThreadExecutionState(0x80000000) 
    print("Sleep mode re-enabled on Windows. sweet dreams")
elif os_name == "posix":
    if platform.system() == "Darwin":
        print("This is a macOS system.")
        caffeinate.terminate()
        print("Caffeinate deactivated on macOS. sweet dreams")
    else:
        print("this is a Linux system.")


This is a windows system.
Sleep mode re-enabled on Windows. sweet dreams
